In [6]:
# import
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import pickle
import warnings
warnings.filterwarnings('ignore')

!pwd
!ls -a ./raw

/Users/hobee/PythonScript/CCF-BDCI-2022-FXFZ
.                    dataA.csv            dataTrain.csv
..                   dataNoLabel.csv      submit_example_A.csv


In [7]:
# load the data
raw_data = pd.read_csv("./cache/label_data.csv")
label_data = pd.read_csv("./cache/label.csv")

In [8]:
def train_model(x,y, model="PlaceHolder"):
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.3,random_state = 33)
    ### 训练模型
    if model == "PlaceHolder":
        model = xgb.XGBClassifier(learning_rate=0.1,
                      n_estimators=100,            # 树的个数--100棵树建立xgboost
                      max_depth=6,                 # 树的深度
                      min_child_weight = 1,        # 叶子节点最小权重
                      gamma=0.,                    # 惩罚项中叶子结点个数前的参数
                      subsample=0.8,               # 随机选择80%样本建立决策树
                      # colsample_btree=0.8,         # 随机选择80%特征建立决策树
                      objective='binary:logistic', # 指定损失函数
                      scale_pos_weight=1,          # 解决样本个数不平衡的问题
                      random_state=27              # 随机数
                      )

    # 拟合
    # model.fit(x_train, y_train, eval_set = [(x_test,y_test)], eval_metric = "auc", early_stopping_rounds = 10,verbose = True)
    model.fit(x_train, y_train)
    
    return model,x_train,x_test,y_train,y_test

In [9]:
# generate the model
gbc = GradientBoostingClassifier(
    n_estimators=50, 
    learning_rate=0.1,
    max_depth=5
)
hgbc = HistGradientBoostingClassifier(
    max_iter=100,
    max_depth=5
)
xgbc = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    n_estimators=100, 
    max_depth=6, 
    learning_rate=0.1
)
cbc = CatBoostClassifier(
    iterations=210, 
    depth=6, 
    learning_rate=0.03, 
    l2_leaf_reg=1, 
    loss_function='Logloss', 
    verbose=0
)
estimators = [
    ('gbc', gbc),
    ('hgbc', hgbc),
    ('xgbc', xgbc),
    ('cbc', cbc)
]
clf = VotingClassifier(
    estimators=estimators, 
    voting="soft"
)

In [10]:
# fit the model
model,x_train,x_test,y_train,y_test = train_model(raw_data,label_data,clf)
model

VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(max_depth=5,
                                                         n_estimators=50)),
                             ('hgbc',
                              HistGradientBoostingClassifier(max_depth=5)),
                             ('xgbc',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='auc', g...
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None,
                                            reg_alpha=None, reg_lambda=None, ...)),
                             ('cbc',
                              <catboost.core.CatBoostClassifier object at 0x17fdfeec0>)],
                 voting='soft')

In [11]:
# save the model
with open("./models/vote_model.pkl", "wb") as file:
    pickle.dump(model,file=file)